# Basics

## Bare

In [ ]:
%cd -q ../../../src/

To access Scrapy Cloud Data, you need to set [Scrapinghub API key](https://app.scrapinghub.com/account/apikey) in `SH_APIKEY` environment variable.

In [ ]:
import arche
from arche import *

The only required parameter is `source`, which accepts various inputs - see signature (`?Arche`) or examples.

In [ ]:
# Reading from json
import json
with open("../docs/source/nbs/data/items_books_1.json") as f:
    raw_items = json.load(f)

In [ ]:
a = Arche(raw_items)

In [ ]:
a = Arche("381798/1/1")

In [ ]:
a.report_all()

We just ran a minimal number of rules. The validation can be improved with adding a json schema, let's infer one from the data we already have.

## JSON schema

In [ ]:
basic_json_schema("381798/1/1")

By itself a basic schema is not very helpful, but you can update it.

In [ ]:
a.source_items.df.head()

Looks like `price` can be checked with regex. Let's also add `category` tag which helps to see the distribution in categoric data and `unique` tag to title to ensure there are no duplicates.

In [ ]:
a.schema = {
    "$schema": "http://json-schema.org/draft-07/schema#",
    "definitions": {
        "float": {
            "pattern": "^-?[0-9]+\\.[0-9]{2}$"
        },
        "url": {
            "pattern": "^https?://(www\\.)?[a-z0-9.-]*\\.[a-z]{2,}([^<>%\\x20\\x00-\\x1f\\x7F]|%[0-9a-fA-F]{2})*$"
        }
    },
    "additionalProperties": False,
    "type": "object",
    "properties": {
        "category": {"type": "string", "tag": ["category"]},
        "price": {"type": "string", "pattern": "^£\d{2}.\d{2}$"},
        "_type": {"type": "string"},
        "description": {"type": "string"},
        "title": {"type": "string", "tag": ["unique"]},
        "_key": {"type": "string"}
    },
    "required": [
        "_key",
        "_type",
        "category",
        "description",
        "price",
        "title"
    ]
}

In [ ]:
a.validate_with_json_schema()

Or if your job is really big you can use almost 100x faster [backend](https://github.com/horejsek/python-fastjsonschema)

In [ ]:
a.glance()

We already got something! Let's execute the whole thing again to see how `category` tag works.

In [ ]:
a.report_all()